#### PART1

In this frst part, we want to randomly retrieve tweets posted by people during the pandemic. The way we use to retrieve tweets
aims at obtaining a sample which is the least possible biased. After retrieving data for each country ( in this first analysis
we start with Serbia and Italy, out homelands, to test the scalability of the code), we want to classify them according to
the topic they present. We are interested in analysing which have been the most discussed topics to see if we get result
similar to those shown in CoronaWiki dataset. We think this is a good way to start analysing how people's interests shifted, how people reacted to the situation and how communication has been affected by COVID-19

In [4]:
# Useful libraries
import pandas as pd
import json
import re
import pickle

# Math libraries
import numpy as np

#Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Natural language processing libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')

# Libray to infer the topics discussed in each tweet
from empath import Empath 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ricca\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TWEETS DATASET CREATION

We start by creating a dataset of retrieved tweets for each analysed country. We decide to save the dataframe in pickle format 
for optimization purposes

In [5]:
# We define lists containing the names and the spoken languages in each country
# france, danimark, germany, italy, netherlands, normway, sweden, serbia, finland, england
countries = ['FR','DK','DE','IT','NL','NO','SE','RS','FI','UK']
languages = ['fr','da','de','it','nl','no','sv','sr','fi','en']
period_per_countries = {}

We use pagelogs data in order to define the period of interest for each country. We start retrieving data from 01/12/2022 since it is reported as the official starting date from the pandemic.

In [6]:
# Importing time series
data_path = './data/'
with open(data_path+'aggregated_timeseries.json','r') as file:
    pagelogs_time_series = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: './data/aggregated_timeseries.json'

In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from datetime import datetime, date, time
import tweepy
import json

In [2]:
bearer_token_balsa = "AAAAAAAAAAAAAAAAAAAAAPXlYgEAAAAAmHO9bfJYAPCZSDa8%2BHELxeAfwgQ%3D91B98esE93293wEGbjH4JsUMe7R3wDok1ZCGNGLLvvQXzNcyBE"
client = tweepy.Client(bearer_token=bearer_token_balsa, wait_on_rate_limit=True)

In [3]:
# Importing time series
data_path = './data/'
with open(data_path+'aggregated_timeseries.json','r') as file:
    time_series = json.load(file)

In [5]:
# Defining period of interest for each country. Dates are retrieved starting from 01/12/2019
for idx,country in enumerate(countries):
    lang = languages[idx]
    dates = [datetime.strptime(date.split()[0], '%Y-%m-%d')  for date in list(time_series[lang]['sum'].keys())]
    dates = [date for date in dates if date.year >= 2020 or (date.year == 2019 and date.month == 12)]
    period_per_countries[country] = dates

In [7]:
# Defining a hour time window to retrieve data
hours = [11,12,13,14,15,16,17,18,19,20]
# We want to give more weights to part of the day closer to dinner / late afternoon -> they will count twice the other
weights = np.ones(len(hours)) / 15
weights[-5:] = weights[-5:]*2

#### Do not run

In [9]:
random_hour = np.random.choice(hours,p = weights)
date = period_per_countries['IT'][100]
start_time = datetime(date.year,date.month,date.day,random_hour)
end_time = datetime(date.year,date.month,date.day,random_hour+2)
tweets = client.search_all_tweets(" COVID lang:en place_country: GB", max_results = 10,
                                     start_time = start_time, end_time = end_time)

In [10]:
for tweet in tweets.data:
    print(tweet.created_at, tweet.text)

None Corona Hysteria Syndrome is actually worse than the virus itself ! 

https://t.co/8qauTPebAI
None Infamous for being a super-spreader of disinformation, could @Nigel_Farage brought COVID-19 onto the @BBCNewsnight set last night? https://t.co/bDUHsd4wdE
None @jamesrbuk Hmm. But, and hear me out, if Jesus is God and God is omnipotent then Jesus IS COVID-19. Admittedly that requires faith in the Holy Trinity.
None Take urgent steps to help the NHS prepare for COVID-19 - Sign the Petition! https://t.co/4JMpPf4Ykj via @UKChange
None ❌🤝

Please note there will be no pre-match handshakes before tonight’s tie with @UCCSoccer due to the Covid-19 outbreak

#CRFC https://t.co/cD2SYXerVA
None With so much hype in the news about Covid-19, I thought a post on fighting off the lurgy would be appropriate.   Here are my top 5 tips on keeping healthy throughout the cold &amp; flu season:

1. Make sure you maintain… https://t.co/UnSC57GnRN
None Self-isolation advice - NHS

Self-isolation advice
- #C

In [11]:
for tweet in tweets.data:
    print(TextBlob(tweet.text).sentiment.polarity)

-0.5
-0.25
0.0
0.0
-0.125
0.26666666666666666
0.0
0.0
0.0
0.0
